В этой работе мы создадим датасет - словарь Старшей Речи из мира Ведьмака.
Выбраны два источника:

- Статья на сайте fandom wiki
- Статья ВК

In [6]:
!pip install requests lxml


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [7]:
import pandas as pd
import requests
import re
from lxml.html import fromstring, tostring, HtmlElement
import pymorphy2

lemmer = pymorphy2.MorphAnalyzer()

Определим датафрейм `word_df` со столбцами "word" и "translation", в который будем записывать одиночные слова и переводы, предобрабатываемые на лету.
И также определим `phrase_df` для устойчивых выражений.

In [18]:
word_df = pd.DataFrame(columns=['text', 'translation'])
phrase_df = pd.DataFrame(columns=['text', 'translation'])

Определим общие функции обработки:

In [19]:
def _root_by_url(url: str) -> HtmlElement:
    """Возвращает объект распарсенной страницы, в котором можно искать элементы по XPATH."""
    return fromstring(requests.get(url).text)


def _add_word(word: str, translation: str) -> None:
    """Добавляет в конец датафрейма `word_df` новую запись."""
    word_df.loc[len(word_df)] = (
        word, 
        _word_to_lemm(translation.replace('ё', 'е')),
    )


def _word_to_lemm(word: str) -> str:
    """Переводит слово в начальную форму."""
    return lemmer.parse(word)[0].normal_form


def _add_phrase(phrase: str, translation: str) -> None:
    """Добавляет в конец датафрейма `phrase_df` новую запись."""
    phrase_df.loc[len(phrase_df)] = (
        phrase, 
        translation.replace('ё', 'е'),
    )

Парсинг статьи Fandom:

In [20]:
# Будем удалять из текста ненужные части.
DELETIONS = [
    '«',
    '»',
    'и т. п.',
    '(?)',
    'р. п.',
]

# Метки, указывающие, что текст не подходит как словарное слово.
STOP_SIGNS = [
    'фонетическое представление',
    'месяц в эльфском календаре',
    'артикль',
    'глагол',
]

# Определим унифицированный разделитель слова и перевода.
SEP = '---SEP---'

root = _root_by_url('https://vedmak.fandom.com/wiki/Старшая_Речь/Лексика')
items = root.xpath('.//div[@id="mw-content-text"]/div[contains(@class, "mw-parser-output")]'
                   '/h2[contains(.,"Фразы и выражения")]/preceding-sibling::ul/li')

count = 0
for item in items:
    item = item.text_content().lower()

    # Проверяем текст на наличие меток о непригодности.
    stop = False
    for sign in STOP_SIGNS:
        if sign in item:
            stop = True
            break

    if stop:
        continue

    # Очищаем текст от лишних подстрок.
    for deletion in DELETIONS:
        item = item.replace(deletion, '')

    # После [ может следовать пояснение, убираем его.
    if '[' in item:
        item = item[:item.find('[')]

    # Устанавливаем общий разделитель (― и — не являются одним символом)
    item = item.replace('―', SEP).replace('—', SEP)

    # Разделителем синонимов может являться / либо ;, а не запятая. Сделаем запятую для унификации.
    item = item.replace('/', ',').replace(';', ',')

    # В конце, как правило, стоит точка. Она нам не нужна.
    item = item.rstrip('.')

    # Формируем всевозможные пары и заносим в датафрейм.
    words, translations = [x.strip().split(',') for x in item.split(SEP, maxsplit=1)]
    for word in words:
        word = word.strip()
        for translation in translations:
            translation = translation.strip()
            _add_word(word, translation)
            count += 1

print(f'В словарь добавлено {count} слов.')

В словарь добавлено 351 слов.


Парсинг статьи ВК:

In [21]:
# Будем удалять из текста ненужные части.
DELETIONS = [
    '1.',
    '2.',
    'эльф(-ы);',
    '(e)',
    '(-e)',
    '“',
    '”',
    ', т.е. объект или субъект, прин-й биосфере',

    '(для ровесников)',
    '(для группы)',
    '(при обращение к более старшему)',
    '.',
    '?',
    '!',
]

# Замены, упрощающие обработку некоторых моментов.
REPLACEMENTS = {
    'cead(-mil)': 'cead,ceadmil',
    '(также va faill)': ',va faill',
    'mint`и': 'mint`u',
    '-': ' ',
    '–': ' ',
    '–': ' ',
    '`': '\'',
    '’': '\'',
    
}

# Метки, указывающие, что текст не подходит как словарное слово.
STOP_SIGNS = [
    'глагол',
    'предлог',
    'частица',
    'артикль',
    'й месяц года',
]

# Здесь разделителем слова и перевода будет являться пробел, слева от которого - латинская буква, а справа - кириллическая.
SEP = r'(?<=[a-z])\s*(?=[а-я])'

root: HtmlElement = _root_by_url('https://vk.com/@thewhitewooolf-starshaya-rech-slovar')
ps: list[HtmlElement] = root.xpath('.//div[@id="article_view_-190268975_57657"]'
                                   '/h2[contains(.,"Выражения и фразы")]/preceding-sibling::p')

word_count = 0
phrase_count = 0
for p in ps:
    # Заменим все <br> на переносы строк для удобства `str.split`
    for br in p.xpath('./br'):
        br.tail = "\n" + br.tail if br.tail else "\n"

    items = p.text_content().split('\n')
    for item in items:
        item = item.lower()

        # Проверяем текст на наличие меток о непригодности.
        stop = False
        for sign in STOP_SIGNS:
            if sign in item:
                stop = True
                break

        if stop:
            continue

        # Очищаем текст от лишних подстрок.
        for deletion in DELETIONS:
            item = item.replace(deletion, '')

        # Производим замены, облегчающие обработку:
        for from_, to in REPLACEMENTS.items():
            item = item.replace(from_, to)

        # После ( может следовать пояснение, убираем его.
        if '(' in item:
            item = item[:item.find('(')]

        # Также после [ может следовать пояснение, убираем его.
        if '[' in item:
            item = item[:item.find('[')]

        # Формируем всевозможные пары и заносим в датафрейм.
        words, translations = [x.strip().split(',') for x in re.split(SEP, item, maxsplit=1)]
        for word in words:
            word = word.strip()
            for translation in translations:
                translation = translation.strip()

                # В одном месте перепутали английскую и русскую "о".
                translation = translation.replace('o', 'о')

                _add_word(word, translation)
                word_count += 1

# ------ Дальше будем парсить отдельный специфичный блок внизу статьи ------

# Здесь разделитель тот же, но порядок обратный.
SEP_REVERSE = r'(?<=[а-я])\s*(?=[a-z])'
cur_sep = SEP_REVERSE

ps: list[HtmlElement] = root.xpath('.//div[@id="article_view_-190268975_57657"]'
                                   '/h2[contains(.,"РАЗГОВОРНИК ПОПУЛЯРНЫХ ФРАЗ")]/following-sibling::p')

for p in ps:
    # Заменим все <br> на переносы строк для удобства `str.split`
    for br in p.xpath('./br'):
        br.tail = "\n" + br.tail if br.tail else "\n"

    items = p.text_content().split('\n')
    for item in items:
        # Начиная с данного слова, снова начинаются обычные слова с обычным порядком, а не фразы. Делаем переход на лету.
        if 'Chodladh' in item:
            cur_sep = SEP

        item = item.lower()

        # Очищаем текст от лишних подстрок.
        for deletion in DELETIONS:
            item = item.replace(deletion, '')

        # Производим замены, облегчающие обработку:
        for from_, to in REPLACEMENTS.items():
            item = item.replace(from_, to)

        # Обрабатываем в зависимости от того фраза это или нет и заносим в датафрейм.
        if cur_sep == SEP:
            # Формируем всевозможные пары и заносим в датафрейм.
            words, translations = [x.strip().split(',') for x in re.split(SEP, item, maxsplit=1)]
            for word in words:
                word = word.strip()
                for translation in translations:
                    translation = translation.strip()
    
                    # В одном месте перепутали английскую и русскую "о".
                    translation = translation.replace('o', 'о')
    
                    _add_word(word, translation)
                    count += 1
            word_count += 1
        else:
            # Не забываем убрать лишние запятые.
            translation, phrase = [x.strip().replace(',', '') for x in re.split(cur_sep, item, maxsplit=1)]
            _add_phrase(phrase, translation)
            phrase_count += 1

print(f'В словарь добавлено {word_count} слов и {phrase_count} устойчивых выражений.')

В словарь добавлено 886 слов и 37 устойчивых выражений.


In [22]:
word_df = word_df.drop_duplicates()

Вывод результатов:

In [23]:
pd.set_option('display.max_rows', 10000)
word_df

,text,translation
0,а,к
1,а,в
2,а,от
3,а,для
4,а,это
5,aab,устье
6,abb,устье
7,aard,гора
8,aard,горный
9,aard,верхний


In [24]:
phrase_df

,text,translation
0,elaine ma'idin,доброе утро
1,tuilleadh ro luath,еще слишком рано
2,tuilleadh luath,еще рано
3,feainne hel'caem n'aen eigean,солнце встанет не скоро
4,breacadh cheana eigean,рассвет уже скоро
5,esseath luath woke,ты рано проснулся
6,n'aen dearme,не спится
7,onas dearme,как спалось с
8,maith,хорошо
9,fath neen woke,почему не разбудили


Мы собрали словарь из 1237 слов и 36 устойчивых выражений.

В дальнейшем при реализации перевода планируется реализовать поиск по синонимам.
К примеру, если надо будет перевести слово "высочайший", которого в словаре нашем нет, то программа должна будет взять синоним, который уже у нас есть - "высший" (aard).

Если и в этом случае поиск будет неудачен, то слово будет пропущено (либо переведено на английский, если хватит сил это реализовать)

In [25]:
word_df.to_csv('./witcher_words.csv', index=False, encoding='utf-8')
phrase_df.to_csv('./witcher_phrases.csv', index=False, encoding='utf-8')